In [1]:
from ipyleaflet import Map, basemaps, Polyline, CircleMarker, Popup, DivIcon, Marker
from ipywidgets import Layout, HTML

import pandas as pd
from pyproj import Geod
import itertools
import json

In [2]:
# Figures out how to draw a curved line between two points
# Makes sure that routes to East Asia/Oceania goes over the pacific

long_cutoff = 80

def get_gcpoints(g, row):
    orig_lat, orig_lon = row['lat_A'], row['lon_A']
    if orig_lon > long_cutoff:
        orig_lon -= 360
    dest_lat, dest_lon = row['lat_B'], row['lon_B']
    if dest_lon > long_cutoff:
        dest_lon -= 360
    gc_points = g.npts(orig_lon, orig_lat, dest_lon, dest_lat, 20)
    revised_gc_points = [(orig_lat, orig_lon)]
    for _lon, _lat in gc_points:
        if _lon > long_cutoff:
            _lon -= 360
        revised_gc_points.append((_lat, _lon))
    revised_gc_points.append((dest_lat, dest_lon))
    return revised_gc_points

    
def generate_map(df, map_name):
    
    g = Geod(ellps="WGS84")
    # m = folium.Map(location=[38,-97], zoom_start=5, tiles="CartoDB dark_matter")
    m = Map(basemap=basemaps.CartoDB.DarkMatter, center=[38,-96], zoom=5, layout=Layout(width='100%', height='800px'))
    
    m.filtered = False
    
    # df = df.sample(n=20, random_state=0).reset_index()    
    
    # Gets a unique list of airports in the data    
    airports = pd.concat([df[['A', 'lat_A', 'lon_A']].set_axis(['code', 'lat', 'lon'], axis=1),
                          df[['B', 'lat_B', 'lon_B']].set_axis(['code', 'lat', 'lon'], axis=1)
                         ]).drop_duplicates('code').reset_index(drop=True)

    
    lines, clearlines, halflines = {}, {}, {}
    hidelines = {}
    for i in range(len(df)):
        print(f'Line {i+1}/{len(df)}', end='\r')
        row = df.loc[i]
        if row['color'] == 'white': continue
        revised_gc_points = get_gcpoints(g, row)
                
        line = Polyline(locations=revised_gc_points, color=row['color'], weight=1, fill=False)
        lines[row['AB']] = line
        m.add_layer(line)

        clearline = Polyline(locations=revised_gc_points, color=row['color'], weight=10, opacity=0, fill=False)
        clearline.popup = HTML(row['AB'])
        clearlines[row['AB']] = clearline
        m.add_layer(clearline)
        
        hideline = Polyline(locations=revised_gc_points, color='black', weight=1, fill=False)
        hidelines[row['AB']] = hideline
    print()

    # Used to make sure that the area around hubs is dominated by the color of the hub, instead of getting covered by other colors
    for i in range(len(df)):
        print(f'Halfline {i+1}/{len(df)}', end='\r')
        row = df.loc[i]
        if row['color'] == 'white': continue
        revised_gc_points = get_gcpoints(g, row)[:5]
        
        halfline = Polyline(locations=revised_gc_points, color = row['color'], weight=1, fill=False)
        halflines[row['AB']] = halfline
        m.add_layer(halfline)
    print()  
    
    def handle_mouseover(**kwargs):
        if m.filtered: return
        
        match = airports.set_index(['lat','lon']).loc[tuple(kwargs['coordinates'])]
        if len(match) != 1:
            return
        match = match[0]
        
        nonstops = [k.split('_') for k,v in lines.items() if match in k]
        nonstops = set(itertools.chain(*nonstops))

        new_layers = [m.layers[0]]
        new_layers += [v for k,v in lines.items() if match in k]
        new_layers += [v for k,v in markers.items() if k in nonstops]
        new_layers += [v for k,v in circles.items() if k in nonstops]
        
        m.layers = new_layers
        m.filtered = True
        
        
    def handle_mouseout(**kwargs):
        if not m.filtered: return
        m.layers = m.old_layers
        m.filtered = False

    # Draws markers over each airport
    circles, markers = {}, {}
    for i in range(len(airports)):
        print(f'Airport {i+1}/{len(airports)}', end='\r')
        row = airports.loc[i]
        
        lat, lon = row['lat'], row['lon']
        if row['lon'] > long_cutoff:
            lon -= 360
        
        circle = CircleMarker(location=[lat,lon], radius=4, color='white', fill_opacity=1)
        circles[row['code']] = circle
        circle.on_mouseover(handle_mouseover)
        circle.on_mouseout(handle_mouseout)
        
        icon = DivIcon(icon_anchor=(0,22), icon_size=(0,0), html=f"<div><p style='color:white'><b>{row['code']}</b></p></div>")
        marker = Marker(location=[lat, lon], icon=icon, draggable=False)
        markers[row['code']] = marker
        
        
        
        # circle.on_mouseover(handle_click)
        m.add_layer(circle)
        m.add_layer(marker)
    print()
        
    m.old_layers = m.layers
    
    m.save(f'python/{map_name}')
    
    circlejson = {'type': 'FeatureCollection', 'features':[]}
    for k,v in circles.items():
        loc = list(reversed(v.location))
        circlejson['features'].append({
            'geometry': {'type': 'Point', 'coordinates': loc},
            'type': 'Feature',
            'id': k
        })
        
    linejson = {'type': 'FeatureCollection', 'features':[]}
    for k,v in lines.items():
        locs = [[lon,lat] for (lat,lon) in v.locations]
        linejson['features'].append({
            'geometry': {'type': 'LineString', 'coordinates': locs},
            'type': 'Feature',
            'id': k,
            'properties': {'style': {'color': v.color}}
        })
    
    with open(f'{map_name}.js', 'w') as f:
        print(f'var airports = {json.dumps(circlejson, indent=2)};\n', file=f)
        print(f'var routes = {json.dumps(linejson, indent=2)};', file=f)
        
    return m

In [3]:
%%time
all_routes = pd.read_csv('data/all.csv')
all_map = generate_map(all_routes, 'all')
# all_map

Line 2063/2063
Halfline 2063/2063
Airport 315/315
CPU times: user 2min 17s, sys: 7.68 s, total: 2min 25s
Wall time: 2min 25s


In [4]:
%%time
aa_routes = pd.read_csv('data/american.csv')
aa_map = generate_map(aa_routes, 'american')
# aa_map

Line 761/761
Halfline 761/761
Airport 231/231
CPU times: user 36.8 s, sys: 2.7 s, total: 39.5 s
Wall time: 41.2 s


In [5]:
%%time
dl_routes = pd.read_csv('data/delta.csv')
dl_map = generate_map(dl_routes, 'delta')
# dl_map

Line 672/672
Halfline 672/672
Airport 216/216
CPU times: user 32.4 s, sys: 2.39 s, total: 34.8 s
Wall time: 35.8 s


In [6]:
%%time
ua_routes = pd.read_csv('data/united.csv')
ua_map = generate_map(ua_routes, 'united')
# ua_map

Line 630/630
Halfline 630/630
Airport 230/230
CPU times: user 33.1 s, sys: 2.54 s, total: 35.7 s
Wall time: 37.6 s
